In [48]:
from PIL import Image
import pywt
import os
import numpy as np
import pandas as pd
from sklearn import preprocessing

In [49]:
df = pd.read_csv("./data/leedsbutterfly_6.csv")
df.shape

(832, 10)

In [50]:
y = np.array([0.895012, 0.878524, 0.902101, 0.905368, 0.618921, 0.777119, 0.832448, 0.891809, 0.765767,  0.80035])
y_expected = np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

(y - y_expected)* (y * (np.ones(len(y)) - y))

array([-0.00986525,  0.09375571,  0.07966886,  0.07756902,  0.14597734,
        0.13460094,  0.11610845,  0.08604682,  0.13735402,  0.12788783])

In [51]:
delta = np.array([
    [-0.00986522, 0.0937556, 0.0796689, 0.0775692,  0.145977,  0.134601,  0.116109, 0.0860468,  0.137354,  0.127888],
    [-0.00986522, 0.0937556, 0.0796689, 0.0775692,  0.145977,  0.134601,  0.116109, 0.0860468,  0.137354,  0.127888],
    [-0.00986522, 0.0937556, 0.0796689, 0.0775692,  0.145977,  0.134601,  0.116109, 0.0860468,  0.137354,  0.127888]
])
wi = np.array([
    [0.846472, 0.846472, 0.846472, 0.846472, 0.846472, 0.846472, 0.846472, 0.846472, 0.846472, 0.846472],
    [0.805749, 0.805749, 0.805749, 0.805749, 0.805749, 0.805749, 0.805749, 0.805749, 0.805749, 0.805749],
    [0.714157, 0.714157, 0.714157, 0.714157, 0.714157, 0.714157, 0.714157, 0.714157, 0.714157, 0.714157]
])

np.multiply(delta, wi)

array([[-0.00835063,  0.07936149,  0.06743749,  0.06566016,  0.12356544,
         0.11393598,  0.09828302,  0.07283621,  0.11626632,  0.10825361],
       [-0.00794889,  0.07554348,  0.06419314,  0.06250131,  0.11762082,
         0.10845462,  0.09355471,  0.06933212,  0.11067285,  0.10304563],
       [-0.00704532,  0.06695622,  0.0568961 ,  0.05539659,  0.1042505 ,
         0.09612625,  0.08292006,  0.06145092,  0.09809232,  0.09133211]])

In [52]:
PATH = './leedsbutterfly/images/'
FILE_NAMES = os.listdir(PATH)

In [53]:
def get_FV(image, it=5):
    LL, (LH, HL, HH) = pywt.dwt2(image, 'haar')
    for _ in range(it):
        LL, (LH, HL, HH) = pywt.dwt2(LL, 'haar')
    return LL.flatten()

In [54]:
cuts = [6]
for c in cuts:
    FVs = []
    for im in os.listdir(PATH):
        image = Image.open(PATH + im).resize((100, 100))
        mt_image = np.asarray(image).astype(int)
        FVs.append(get_FV(mt_image, c).tolist())
    cols = len(FVs[0])
    df = pd.DataFrame(FVs, columns=[str(_) for _ in range(cols)])
    x = df.values #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df = pd.DataFrame(x_scaled)
    
    df.to_csv(f'./data/leedsbutterfly_{c}.csv', index=False)

In [55]:
print(np.unique([int(FILE_NAMES[_][0:3]) for _ in range(len(FILE_NAMES))]))

[ 1  2  3  4  5  6  7  8  9 10]


In [56]:
y_expected = []

for _ in range(len(FILE_NAMES)):
    arr = np.zeros(10).astype(int)
    arr[int(FILE_NAMES[_][0:3])-1] = 1
    y_expected.append(list(arr))

y_csv = pd.DataFrame(y_expected, columns=[str(_) for _ in range(10)])
y_csv.to_csv('./data/y_expected.csv', index=False)

In [57]:
from random import shuffle

In [58]:
c = 6
y_csv = pd.read_csv(f'./data/y_expected.csv')
df = pd.read_csv(f'./data/leedsbutterfly_{c}.csv')

In [59]:
len(y_csv)

832

In [60]:
print(y_csv.head(5))
print(df.head(5))

   0  1  2  3  4  5  6  7  8  9
0  1  0  0  0  0  0  0  0  0  0
1  1  0  0  0  0  0  0  0  0  0
2  1  0  0  0  0  0  0  0  0  0
3  1  0  0  0  0  0  0  0  0  0
4  1  0  0  0  0  0  0  0  0  0
          0         1         2         3         4         5         6  \
0  0.553845  0.579971  0.605285  0.613381  0.609980  0.598897  0.595205   
1  0.002022  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2  0.999908  0.992525  0.938457  0.939384  0.960945  0.947212  0.939576   
3  0.999632  0.991866  0.938848  0.944179  0.967341  0.952911  0.945764   
4  0.343007  0.357721  0.361068  0.363442  0.375337  0.390464  0.401884   

          7         8         9  ...        90        91        92        93  \
0  0.596910  0.603278  0.613988  ...  0.531139  0.514343  0.512148  0.524194   
1  0.001478  0.011321  0.027924  ...  0.055686  0.017960  0.000000  0.003280   
2  0.935526  0.936377  0.889932  ...  0.929608  0.925141  0.616847  0.691343   
3  0.942420  0.941946  0.942309  ... 

In [61]:
ind_list=[i for i in range(len(y_csv))]
shuffle(ind_list)

In [62]:
ind_list

[499,
 245,
 760,
 556,
 590,
 324,
 517,
 493,
 89,
 492,
 575,
 781,
 553,
 21,
 148,
 526,
 171,
 502,
 175,
 692,
 203,
 806,
 333,
 802,
 116,
 95,
 144,
 596,
 270,
 154,
 210,
 812,
 533,
 714,
 489,
 353,
 269,
 31,
 253,
 264,
 111,
 227,
 45,
 467,
 660,
 43,
 476,
 128,
 22,
 444,
 555,
 783,
 74,
 330,
 439,
 455,
 385,
 826,
 593,
 736,
 782,
 665,
 581,
 522,
 723,
 318,
 108,
 28,
 191,
 512,
 326,
 188,
 684,
 610,
 820,
 754,
 223,
 821,
 197,
 621,
 810,
 641,
 290,
 132,
 10,
 147,
 548,
 334,
 759,
 474,
 69,
 487,
 594,
 345,
 547,
 732,
 426,
 730,
 37,
 84,
 271,
 564,
 466,
 800,
 765,
 735,
 25,
 62,
 332,
 469,
 603,
 728,
 364,
 149,
 263,
 229,
 541,
 530,
 797,
 92,
 276,
 733,
 24,
 699,
 311,
 278,
 796,
 87,
 431,
 358,
 102,
 138,
 297,
 437,
 816,
 103,
 362,
 88,
 671,
 79,
 57,
 538,
 122,
 453,
 710,
 38,
 601,
 412,
 655,
 94,
 121,
 418,
 552,
 150,
 14,
 514,
 331,
 285,
 372,
 156,
 571,
 727,
 240,
 23,
 56,
 308,
 181,
 513,
 612,
 563,
 443,


In [63]:
y_csv = y_csv.iloc[ind_list]
df = df.iloc[ind_list]

In [64]:
print(y_csv.head(5))
print(df.head(5))

     0  1  2  3  4  5  6  7  8  9
499  0  0  0  0  0  1  0  0  0  0
245  0  0  0  1  0  0  0  0  0  0
760  0  0  0  0  0  0  0  0  0  1
556  0  0  0  0  0  0  1  0  0  0
590  0  0  0  0  0  0  1  0  0  0
            0         1         2         3         4         5         6  \
499  0.197595  0.476141  0.563442  0.549502  0.546967  0.542831  0.538320   
245  0.993781  0.969746  0.954197  0.939874  0.926149  0.869945  0.818007   
760  0.336994  0.336634  0.338097  0.328117  0.317479  0.311098  0.293896   
556  0.971890  0.903592  0.842716  0.792210  0.744003  0.699142  0.659283   
590  0.472281  0.469049  0.466682  0.469278  0.479718  0.494424  0.501088   

            7         8         9  ...        90        91        92  \
499  0.533011  0.527890  0.522429  ...  0.292643  0.292575  0.294205   
245  0.786440  0.758460  0.717113  ...  0.844756  0.844562  0.849966   
760  0.258010  0.225584  0.220481  ...  0.290520  0.301315  0.335782   
556  0.625122  0.595427  0.565759  ...  0.581

In [65]:
df.to_csv(f'./data/leedsbutterfly_{c}.csv', index=False)
y_csv.to_csv('./data/y_expected.csv', index=False)

In [67]:
ls = [0.0983208, 0.105734, 0.0982891, 0.0996633, 0.103204, 0.0959113, 0.0992619, 0.0976498, 0.0999195, 0.102046]
print(sum(ls))
ls.index(max(ls))

0.9999996999999999


1

In [69]:
ls = [0.0886512, 0.0845005, 0.09336, 0.0747104, 0.0981759, 0.118552, 0.141167, 0.0817872, 0.106676, 0.112421]
print(sum(ls))
print(ls.index(max(ls)))
print(max(ls))

1.0000011999999998
6
0.141167
